In [1]:
#| default_exp 07_parse-wikipedia-abstract

In [2]:
#| hide
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [3]:
#| export
import os, bz2, queue, wikitextparser, multiprocessing, xml.sax, io, argparse, gzip
from timeit import default_timer as timer

In [4]:
#| export
def process_worker(articles_source):
    with gzip.open(articles_source, 'rb') as filehandler:
        contents = filehandler.read()
    return process_contents(contents)

def process_contents(contents):
    reader = XMLSAXParser()
    try:
        byte_stream = io.BytesIO(contents)
        xml.sax.parse(byte_stream, reader)
    finally:
        byte_stream.close()
        return reader.all_contents

In [5]:
#| export
class XMLSAXParser(xml.sax.ContentHandler):
    def __init__(self):
        super().__init__()

        self.read_stack = []
        self.page_title = None
        self.page_content = None

        self.page_count = 0
        self.in_page = False

        self.all_contents = dict()

    def startElement(self, tag_name, attributes):

        self.text_aggregate = []

        if tag_name == "doc":
            self.page_title = None
            self.page_content = None
            self.in_page = True

        self.read_stack.append(tag_name)

    def endElement(self, tag_name):
        if (len(self.read_stack) > 0) and (tag_name == self.read_stack[-1]):
            del self.read_stack[-1]
        else:
            raise Exception("Tag ({}) does not match open tag ({}).".format(tag_name, self.read_stack[-1]))

        element_string = ''.join(self.text_aggregate)

        if tag_name == "doc":
            self.in_page = False
            self.all_contents[self.page_title] = self.page_content
            print(f'Processed: {self.page_title}')
        else:
            if self.in_page:
                if self.read_stack[-1] == "doc":
                    if tag_name == "title":
                        self.page_title = element_string
                    elif (tag_name == "abstract") and self.read_stack[-1]:
                        self.page_content = element_string

    text_aggregate = []

    def characters(self, content):
        if self.in_page:
            self.text_aggregate.append(content)
            

In [6]:
#| export
def main(articles_source, cache_dir):
    os.makedirs(cache_dir, exist_ok=True)
    contents = process_worker(articles_source)

    with open(f'{cache_dir}/abstract.txt', 'w') as file:
        for k,v in contents.items():
            file.write(f'{k}->{v}\n')

def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--articles_source', type=str, required=True)
    parser.add_argument('--cache_dir', type=str, required=True)
    return parser.parse_args()

In [ ]:
#| export
if __name__ == "__main__":
    start_time = timer()

    args = parse_args()

    main(args.articles_source, args.cache_dir)

    end_time = timer()
    print(f'Time elapsed: {end_time-start_time:.2f} seconds.')